# Create DB

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from utils import *

db_name = 'esj2.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

with open('sqls/init1.sql', 'r') as sql_file:
        sql_script = sql_file.read()
    
cursor.executescript(sql_script)
conn.commit()
conn.close()

# Populate DB

### PROMOTION - 80, CUSTOMER - 3000, MOVIE - 500, ARTIST - 2000, CAST - 5000, SESSION - 10K

In [3]:
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Promotion
df = pd.read_csv('toyData/promotions.csv')
halls = [(
            df['name'].iloc[i], 
            int(df['discount'].iloc[i]),
            df['start'].iloc[i],
            df['end'].iloc[i],
        ) for i in range(df.shape[0])]

sql_query_template = """
    INSERT INTO PROMOTION (name, discount, start, end)
    VALUES (?, ?, ?, ?);
"""

for hall in halls:
    cursor.execute(sql_query_template, hall)
    
# Customer
df = pd.read_csv('toyData/customer.csv')
halls = [(
            df['name'].iloc[i], 
            df['dob'].iloc[i], 
            df['address'].iloc[i],
            df['gender'].iloc[i],
        ) for i in range(df.shape[0])]

sql_query_template = """
    INSERT INTO CUSTOMER (name, dob, address, gender)
    VALUES (?, ?, ?, ?);
"""

for hall in halls:
    cursor.execute(sql_query_template, hall)


# Movies
df = pd.read_csv('toyData/movies.csv')
halls = [(
            df['title'].iloc[i], 
            df['country'].iloc[i], 
            df['language'].iloc[i], 
            round(df['cost'].iloc[i]*1000000), 
            df['releaseDate'].iloc[i], 
            df['genre'].iloc[i]
        ) for i in range(df.shape[0])]

sql_query_template = """
    INSERT INTO MOVIE (title, country, language, cost, releaseDate, genre)
    VALUES (?, ?, ?, ?, ?, ?);
"""

for hall in halls:
    cursor.execute(sql_query_template, hall)

# Artists
df = pd.read_csv('toyData/artist.csv')
halls = [(
            df['name'].iloc[i], 
            df['dob'].iloc[i], 
            df['gender'].iloc[i],
        ) for i in range(df.shape[0])]

sql_query_template = """
    INSERT INTO ARTIST (name, dob, gender)
    VALUES (?, ?, ?);
"""

for hall in halls:
    cursor.execute(sql_query_template, hall)

# Cast
# Directors
halls = [(i, 
          random.randint(1, 100), 
          1,) for i in range(1, 500)]

sql_query_template = """
    INSERT INTO CAST (movieId, artistId, isDirector)
    VALUES (?, ?, ?);
"""

for hall in halls:
    cursor.execute(sql_query_template, hall)

# Actors
halls = [(random.randint(1, 500), 
          random.randint(101, 2000), 
          "1" if random.random() < 0.05 else "0") for i in range(4500)]

sql_query_template = """
    INSERT INTO CAST (movieId, artistId, isDirector)
    VALUES (?, ?, ?);
"""

for hall in halls:
    cursor.execute(sql_query_template, hall)

# Sessions
df = pd.read_csv('toyData/sessions.csv')

sql_query_template = """
    INSERT INTO SESSION (dateTime, movieId, hall, size, state, city, address)
    VALUES (?, ?, ?, ?, ?, ?, ?);
"""

for i in range(df.shape[0]):
    hall = (df['dateTime'].iloc[i],
            int(df['movieId'].iloc[i]),
            int(df['hall'].iloc[i]),
            int(df['size'].iloc[i]),
            df['state'].iloc[i],
            df['city'].iloc[i],
            df['address'].iloc[i]
           )
    
    cursor.execute(sql_query_template, hall)

conn.commit()
cursor.close()
conn.close()

### Ticket Transaction - 1M, Ticket - Approx 3M

In [4]:
from utils import *
import sqlite3

conn = sqlite3.connect(db_name)
cursor = conn.cursor()

sql_transaction_template = """
    INSERT INTO TICKET_TRANSACTION (dateTime, totalPrice, payMethod, customerId, promotionId, os, browser, sessionId, numTickets)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

sql_ticket_template = """
    INSERT INTO TICKET (price, row, seat, transactionId, sessionId)
    VALUES (?, ?, ?, ?, ?);
"""

df_session = pd.read_csv('toyData/sessions.csv')


for i in range(1000000):
    number_of_tickets = random.randint(1, 5)
    ticket_price = random.choices([15, 30, 50], weights=[0.4, 0.4, 0.2])[0]
    
    row = random.randint(1, 20)
    seat = random.randint(1, 20)
    session = random.randint(1, df_session.shape[0])
    
    payment = random.choices(["Credit Card", "Wire Transfer", "Cash"], weights=[0.6, 0.1, 0.3])[0]
    if payment != "Cash":
        os = random.choices(["Windows", "Mac", "Linux"], weights=[0.5, 0.4, 0.1])[0]
        browser = random.choices(["Chrome", "Web Browser", "Safari"], weights=[0.7, 0.2, 0.1])[0]
    else:
        os = None
        browser = None
    
    hall = (random_transaction_time(df_session['dateTime'].iloc[session-1]),
            ticket_price*number_of_tickets,
            payment,
            None if random.random() < 0.9 else random.randint(1, 3000),
            None,
            os,
            browser,
            session,
            number_of_tickets,
           )
    
    cursor.execute(sql_transaction_template, hall)
    
    for j in range(number_of_tickets):
        hall = (ticket_price,
                row,
                seat+j,
                i,
                session,
               )
        cursor.execute(sql_ticket_template, hall)

conn.commit()
cursor.close()
conn.close()

### Connect promotion

In [5]:
import sqlite3

conn = sqlite3.connect(db_name)
cursor = conn.cursor()

cursor.execute("SELECT id, start, end, discount FROM PROMOTION")
promotions = cursor.fetchall()

for promotion in promotions:
    promo_id, start_date, end_date, discount_amount = promotion
    cursor.execute("SELECT COUNT(*) FROM TICKET_TRANSACTION WHERE dateTime BETWEEN ? AND ?", 
                   (start_date, end_date))
    total_rows = cursor.fetchone()[0]
    rows_to_update = int(0.4 * total_rows)
    
    cursor.execute("SELECT id, totalPrice FROM TICKET_TRANSACTION WHERE dateTime BETWEEN ? AND ? ORDER BY RANDOM() LIMIT ?",
                   (start_date, end_date, rows_to_update))
    rows_to_update = cursor.fetchall()
    
    for row in rows_to_update:
        ticket_id, original_price = row
        discounted_price = original_price - discount_amount
        cursor.execute("UPDATE TICKET_TRANSACTION SET promotionId = ?, totalPrice = ? WHERE id = ?", 
                       (promo_id, discounted_price, ticket_id))

conn.commit()
conn.close()

# Index DB

In [6]:
import sqlite3

conn = sqlite3.connect(db_name)
cursor = conn.cursor()

index_commands = [
    "CREATE INDEX idx_promotion_date ON PROMOTION(start, end);",
    "CREATE INDEX idx_customer_dob ON CUSTOMER(dob);",
    "CREATE INDEX idx_ticket_transactionId ON TICKET(transactionId);",
    "CREATE INDEX idx_ticket_sessionId ON TICKET(sessionId);",
    "CREATE INDEX idx_movie_releaseDate_genre ON MOVIE(releaseDate, genre);",
    "CREATE INDEX idx_cast_movieId ON CAST(movieId);",
    "CREATE INDEX idx_cast_artistId ON CAST(artistId);",
    "CREATE INDEX idx_artist_dob ON ARTIST(dob);",
    "CREATE INDEX idx_session_dateTime_movieId ON SESSION(dateTime, movieId);",
    "CREATE INDEX idx_ticket_transaction_customerId ON TICKET_TRANSACTION(customerId);",
    "CREATE INDEX idx_ticket_transaction_promotionId ON TICKET_TRANSACTION(promotionId);",
    "CREATE INDEX idx_ticket_transaction_sessionId ON TICKET_TRANSACTION(sessionId);"
]

for command in index_commands:
    cursor.execute(command)

conn.commit()
conn.close()

print("Indexes created successfully.")


Indexes created successfully.


# crop image

In [5]:
from PIL import Image

def crop_right_half(image_path, output_path):
    # Open the image
    img = Image.open(image_path)
    
    # Get dimensions of the image
    width, height = img.size
    
    # Crop the right half of the image
    cropped_img = img.crop((0, 0, width // 2, height))
    
    # Save the cropped image
    cropped_img.save(output_path)

# Example usage:
for i in range(1, 9):
    input_image_path = f"image-00{i}.png"
    output_image_path = f"cropped-image-00{i}.png"
    crop_right_half(input_image_path, output_image_path)
